Gestione Orari

In [26]:
import pytz
from datetime import datetime

italy = pytz.timezone('Europe/Rome')

now = italy.localize(datetime.now())
print(now)
print(now.tzinfo)

alarm_str = sensor.poco_f1_next_alarm

if (alarm_str != 'unavailable'):

    alarm = datetime.strptime(sensor.poco_f1_next_alarm, '%Y-%m-%dT%H:%M:%S%z')

    alarm = alarm.replace(tzinfo=pytz.utc).astimezone(italy)

    print(alarm)
    print(alarm.tzinfo)

else:

    print('No available Alarm')

2022-10-06 10:27:19.910592+02:00
Europe/Rome
2022-10-07 08:30:00+02:00
Europe/Rome


data TEST

In [ ]:
from datetime import datetime, timedelta
from time import time

import pytz

def setup_chargers(**kwargs):

    battery = int(state.get(kwargs.get('battery')))
    charging = state.get(kwargs.get('charging_state')) == "on"
    charger = kwargs.get('charger')
    start = datetime.strptime(kwargs.get('day_start_time'), '%H:%M:%S').time()
    end = datetime.strptime(kwargs.get('day_end_time'), '%H:%M:%S').time()

    alarm = state.get(kwargs.get('alarm_time'))
    if alarm != 'unavailable':
        alarm = datetime.strptime(alarm, '%Y-%m-%dT%H:%M:%S%z')
        alarm = alarm.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('Europe/Rome'))

    print("battery level: " + str(battery))
    print("is charging: " + str(charging))
    print("day start time: " + str(start))
    print("day end time: " + str(end))
    print("charger name: " + charger + " state: " + str(state.get(charger)))
    print("alarm_state: " + str(alarm))


kwargs = dict({
    'battery': 'sensor.poco_f1_battery_level', 
    'charging_state': 'binary_sensor.poco_f1_is_charging', 
    'charger': 'switch.caricatore', 
    'alarm_time': 'sensor.poco_f1_next_alarm', 
    'day_start_time': '08:00:00', 
    'day_end_time': '23:00:00'
    })

setup_chargers(**kwargs)



: 

Night Charge TEST

In [41]:
from datetime import datetime, timedelta
from time import time

def night_charge(start=None, end=None, alarm=None, charger=None, battery=None, debug=None): #debug is used for setting an arbitrary actual time leave None for use the actual time
    # switching start and end time so now start will be the start of the night charge and end its end
    temp = start
    start = end
    end = temp

    print("start: " + str(start) + " end: " + str(end) + " alarm: " + alarm + " charger: " + charger + " battery: " + battery)

    if (start == None or charger == None or battery == None or not(end != None or alarm != None)):
        print("not enough data to start the ngiht charge, exit")
        return
    
    if debug == None:

        italy = pytz.timezone('Europe/Rome')
        now = italy.localize(datetime.now())
    
    else: 
        now = debug

    start = now.replace(hour=start.hour, minute=start.minute, second=start.second)
    end = now.replace(hour=end.hour, minute=end.minute, second=end.second)

    if now < start: 
        start = start - timedelta(days=1)
        print('After Midnight')
    
    if end < start:
        end = end + timedelta(days=1)
        print('Before Midnight')
    
    if alarm != None and state.get(alarm) != 'unavailable':
        alarm = datetime.strptime(state.get(alarm), '%Y-%m-%dT%H:%M:%S%z')
        alarm = alarm.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('Europe/Rome'))
        end = alarm

    if not(start <= now < end):
        print("Error not right time, now: " + str(now) + " start: " + str(start) + " end: " + str(end))
        return
    
    print("Starting night charge, now: " + str(now) + " start: " + str(start) + " end: " + str(end))

    return

debug = pytz.timezone('Europe/Rome').localize(datetime.now())
debug = debug.replace(hour=2, minute=30, second=0)

night_charge(datetime.strptime("08:00:00", '%H:%M:%S').time(), datetime.strptime("23:00:00", '%H:%M:%S').time(), "sensor.poco_f1_next_alarm", "switch.caricatore", "sensor.poco_f1_battery_level", None)

start: 23:00:00 end: 08:00:00 alarm: sensor.poco_f1_next_alarm charger: switch.caricatore battery: sensor.poco_f1_battery_level
After Midnight
Error not right time, now: 2022-10-14 15:15:49.932555+02:00 start: 2022-10-13 23:00:00.932555+02:00 end: 2022-10-14 08:00:00.932555+02:00


Service caller modules

In [28]:
import service_caller as sc

sc.send_telegram_notification('test')

In [6]:
import service_caller as sc

charger = "switch.caricatore"

#sc.switch_turn_on(charger)
sc.switch_turn_off(charger)

Turning off switch id = switch.caricatore
